# Audio → Score (Colab Entry)
이 노트북은 Colab에서 본 프로젝트를 바로 실행하기 위한 셀 모음입니다.
- 전제: 이 노트북이 저장소 루트의 `notebooks/` 폴더 안에 있고, 상위 폴더가 프로젝트 루트입니다.
- 흐름: 런타임 확인 → 의존성 설치 → 입력/출력 폴더 준비 → 샘플 오디오 준비 → 파이프라인 실행 → 산출물 확인/다운로드.
- MLflow 아티팩트는 기본적으로 `./mlruns` 아래에 저장됩니다.


In [ ]:
# 1) 런타임/환경 확인 및 작업 디렉토리 이동
import os, sys, platform, subprocess, pathlib
print('Python:', sys.version)
print('Platform:', platform.platform())
try:
    print(subprocess.check_output(['nvidia-smi']).decode())
except Exception:
    print('nvidia-smi not available (CPU runtime).')

# 이 노트북이 notebooks/ 안에 있다고 가정하고, 프로젝트 루트로 이동
nb_cwd = pathlib.Path.cwd()
if nb_cwd.name == 'notebooks' and (nb_cwd.parent / 'requirements.txt').exists():
    os.chdir(nb_cwd.parent.as_posix())
    print('Changed working dir to repo root:', os.getcwd())
else:
    # 휴리스틱: 현재/내용 경로들에서 프로젝트 루트 탐색
    for cand in [pathlib.Path('.'), pathlib.Path('/content'), pathlib.Path('/content/drive/MyDrive')]:
        if (cand / 'requirements.txt').exists() and (cand / 'scripts' / 'infer_pipeline.py').exists():
            os.chdir(cand.as_posix())
            print('Detected project root:', os.getcwd())
            break
print('CWD:', os.getcwd())


In [ ]:
# 2) 필수 패키지 설치 (프로젝트 requirements + 추가 전사/분리 라이브러리)
# Colab에서는 !pip 사용. 이미 설치된 항목은 재사용됨.
%pip install -q -r requirements.txt || echo 'requirements.txt 기반 설치 일부 실패 또는 파일 없음'
%pip install -q demucs basic-pitch music21 pretty_midi transformers hydra-core mlflow librosa soundfile torchaudio --upgrade

import importlib, sys
needed = ['demucs','basic_pitch','music21','pretty_midi','transformers','librosa','soundfile','torchaudio','mlflow']
missing = []
for m in needed:
    try:
        importlib.import_module(m)
    except Exception as e:
        missing.append((m,str(e)))
print('Missing modules:', missing if missing else 'None (all imported)')

# Torch & Device 정보
import torch
print('Torch version:', torch.__version__)
print('CUDA available:', torch.cuda.is_available())
if torch.cuda.is_available():
    print('CUDA device count:', torch.cuda.device_count())


In [ ]:
# 3) 입력/출력/임시/MLflow 폴더 준비
import os, pathlib
INPUT_DIR = './inputs'
OUTPUT_DIR = './colab_outputs'
MLRUNS_DIR = './mlruns'
for d in [INPUT_DIR, OUTPUT_DIR, MLRUNS_DIR]:
    os.makedirs(d, exist_ok=True)
print('Dirs ready:', INPUT_DIR, OUTPUT_DIR, MLRUNS_DIR)

# MLflow 파일 기반 tracking 설정
import mlflow
mlflow.set_tracking_uri(f'file:{MLRUNS_DIR}')
print('MLflow tracking URI set to', mlflow.get_tracking_uri())


In [ ]:
# 4) 샘플 오디오 준비 (업로드 + 다운로드 옵션)
from google.colab import files
print('로컬 WAV 업로드 (선택)')
uploaded = files.upload()  # 사용자가 선택 가능
for fn, data in uploaded.items():
    dest = f'{INPUT_DIR}/{fn}'
    with open(dest, 'wb') as f:
        f.write(data)
    print('Saved', dest)

# 공개 무음/짧은 샘플(대체):
import os
os.system(f"gdown -q -O {INPUT_DIR}/sample.wav https://cdn.jsdelivr.net/gh/anars/blank-audio/1-second-of-silence.wav")
print('Sample file at', f'{INPUT_DIR}/sample.wav')

import glob
print('Input listing:', glob.glob(f'{INPUT_DIR}/*.wav'))


In [ ]:
# 5) Hydra 파이프라인 실행 (scripts/infer_pipeline.py 사용)
# 기본 config 파일 위치와 override 예시를 보여줍니다.
import os, subprocess, shlex, time, glob, json

DEFAULT_INPUT = f'{INPUT_DIR}/sample.wav'
# 사용자가 업로드한 첫 번째 파일이 있으면 그것으로 대체
user_wavs = [p for p in glob.glob(f'{INPUT_DIR}/*.wav') if os.path.basename(p) != 'sample.wav']
if user_wavs:
    DEFAULT_INPUT = user_wavs[0]
print('Using input:', DEFAULT_INPUT)

# 실행 명령 구성 (Hydra override 예시)
cmd = f"python scripts/infer_pipeline.py pipeline.io.input_path={DEFAULT_INPUT} pipeline.io.output_dir={OUTPUT_DIR}"
print('Run command:', cmd)
start = time.time()
proc = subprocess.run(shlex.split(cmd), capture_output=True, text=True)
print('Return code:', proc.returncode)
print('STDOUT:\n', proc.stdout[:2000])
print('STDERR:\n', proc.stderr[:2000])
print('Elapsed sec:', time.time()-start)

# 간단 산출물 나열
midi_files = glob.glob(f'{OUTPUT_DIR}/**/*.mid*', recursive=True)
xml_files = glob.glob(f'{OUTPUT_DIR}/**/*.xml*', recursive=True)
print('MIDI files:', midi_files)
print('MusicXML files:', xml_files)

# 메타/로그 JSON (있다면) 출력 일부
meta_jsons = glob.glob(f'{OUTPUT_DIR}/**/*.json', recursive=True)
for mj in meta_jsons[:3]:
    with open(mj,'r') as f:
        print('---', mj)
        print(f.read()[:400])


In [ ]:
# 6) 산출물 확인 및 다운로드 도우미
from google.colab import files
import glob
midi_files = glob.glob(f'{OUTPUT_DIR}/**/*.mid*', recursive=True)
xml_files = glob.glob(f'{OUTPUT_DIR}/**/*.xml*', recursive=True)
print('MIDI:', midi_files)
print('XML:', xml_files)

AUTO_DOWNLOAD = False  # True로 바꾸면 자동 다운로드
if AUTO_DOWNLOAD:
    for f in midi_files + xml_files:
        try:
            files.download(f)
        except Exception as e:
            print('Download failed for', f, e)


In [ ]:
# 7) 재실행(새 파일 업로드 → 즉시 파이프라인 실행)
from google.colab import files
import subprocess, os
print('새로운 WAV 파일을 선택하면 자동으로 처리합니다.')
uploaded = files.upload()
for fn, data in uploaded.items():
    ipath = os.path.join('inputs', fn)
    with open(ipath, 'wb') as f:
        f.write(data)
    print('Saved', ipath)
    cmd = ['python','-m','scripts.infer_pipeline', f'pipeline.io.input_path={ipath}', f'pipeline.io.output_dir={OUTPUT_DIR}']
    print('Running:', ' '.join(cmd))
    subprocess.run(cmd, check=False)


## 참고: 런타임 재시작 후 최소 실행 순서
1) "1) 런타임/환경 확인" 셀 실행하여 작업 디렉토리를 루트로 이동
2) "2) 필수 패키지 설치" 셀 실행
3) "3) 폴더 준비" 셀 실행
4) "4) 샘플/업로드" 셀 실행(필요 시)
5) "5) 파이프라인 실행" 셀 실행
6) "6) 산출물 확인" 셀로 다운로드

문제 발생 시:
- CUDA 메모리 부족: 런타임을 T4/V100 등 GPU로 변경하거나, 입력 길이를 30~60초로 줄이세요.
- demucs 모델 다운로드 실패: Colab 런타임 네트워크 재시도 또는 모델 이름을 바꿔보세요(htdemucs 등).
- transformers/torch 버전 충돌: 두 번째 셀에서 특정 버전으로 고정 설치 후 런타임 재시작을 고려하세요.
